In [1]:
%load_ext tensorboard

import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Conv2DTranspose, concatenate, Flatten, Reshape
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
import pandas as pd
import numpy as np
from PIL import Image
from math import floor
import matplotlib.pyplot as plt
import datetime
from UAVSequence import UAVSequence
from ValidationCallback import ValidationCallback
from VAEModel import Sampling, VAE
# Clear logs from previous runs
# %rm -rf ../logs/

/home/pranavr/ego-planner/src/neural_net/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pranavr/ego-planner/src/neural_net/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pranavr/ego-planner/src/neural_net/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pranavr

In [2]:
NUM_INPUTS = 13 # used to be 13
NUM_OUTPUTS = 9

img_shape = (480, 640)

# Load the dataset input and output
x = np.loadtxt('../dataset/0_input.csv', delimiter=',')#, usecols=tuple(range(NUM_INPUTS))) # (num training points, 13)
y = np.loadtxt('../dataset/0_output.csv', delimiter=',', usecols=tuple(range(NUM_OUTPUTS))) # (num training points, 11)

# Shuffle x and y, but store the indices so we can still match them with the images
hash_table = np.random.permutation(len(x))

x = x[hash_table]
y = y[hash_table]

# X = np.zeros((len(inputNums), *img_shape, 1)) # (num training points, 480, 640)

# For each of the training examples there should be exactly 1 image
# for i in range(0, len(inputNums)):
#     image = Image.open('../dataset/img_' + str(i) + '.jpg') 
#     imageArr = np.asarray(image).reshape(*img_shape, 1) # Converts the image into a 2D 480*640 array

#     X[i] = imageArr / 255


# img_data = ImageDataGenerator(rescale=1/255)
# img_data.flow_from_directory('../dataset', target_size=img_shape, color_mode='grayscale')


In [3]:
max_mag = 2.0

# print(inputNums.shape);

# def mag(x, y):
#     return np.sqrt(x * x + y * y)

# for inputRow in inputNums:
#     m = mag(*inputRow[0:2]) # constrain goal positions
#     if m > max_mag:
#         inputRow[0] = inputRow[0] * max_mag / m 
#         inputRow[1] = inputRow[1] * max_mag / m 

Let's view some of our data images

In [4]:
num_show = 10

# fig = plt.figure(figsize=(480/20, 640/20), dpi=100)
# for i in range(num_show):
#     ax = fig.add_subplot(num_show, 1, i+1)
#     plt.imshow(X[i])

Next comes actually training on the data. 
PS. If you want to view the tensorboard data, run `tensorboard --logdir logs/fit` in another window.

In [5]:
numerical_inputs = Input(shape=(NUM_INPUTS,))
# image_inputs = Input(shape=(*img_shape, 1))

VALIDATION_SPLIT = 0.2
xy = UAVSequence(x, y, 32, img_shape, hash_table, VALIDATION_SPLIT, False)
xy_validation = UAVSequence(x, y, 32, img_shape, hash_table, VALIDATION_SPLIT, True)


latent_dim = 20

encoder_inputs = Input(shape=(*img_shape, 1))
x = Conv2D(32,  3, strides=4, padding="same", activation="relu")(encoder_inputs)
x = Conv2D(64,  3, strides=4, padding="same", activation="relu")(x)
x = Conv2D(128, 3, strides=2, padding="same", activation="relu")(x)
x = Conv2D(128, 3, strides=2, padding="same", activation="relu")(x)
x = Conv2D(256, 3, strides=2, padding="same", activation="relu")(x)
x = Flatten()(x)
x = Dense(50, activation="relu")(x)
z_mean = Dense(latent_dim, name="z_mean")(x)
z_log_var = Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

latent_inputs = Input(shape=(latent_dim,))
x = Dense(15 * 20 * 32, activation="relu")(latent_inputs)
x = Reshape((15, 20, 32))(x)
x = Conv2DTranspose(256, 3, strides=2, padding="same", activation="relu")(x)
x = Conv2DTranspose(128, 3, strides=2, padding="same", activation="relu")(x)
x = Conv2DTranspose(128, 3, strides=2, padding="same", activation="relu")(x)
x = Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
x = Conv2DTranspose(32, 3, activation="relu", strides=1, padding="same")(x)

decoder_outputs = Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

# Process the image data first
# x_image = Conv2D(32, (7, 7), strides=(4, 4), padding='same', activation='relu')(image_inputs)
# x_image = Conv2D(64, (5, 5), strides=(4, 4), padding="same", activation="relu")(x_image)
# x_image = Conv2D(128, (3, 3), strides=(2, 2), padding="same", activation="relu")(x_image)
# x_image = Conv2D(128, (3, 3), strides=(2, 2), padding="same", activation="relu")(x_image)
# x_image = Conv2D(256, (3, 3), strides=(2, 2), padding="same", activation="relu")(x_image)
# x_image = Model(image_inputs, x_image)
# x_image = Flatten()(x_image.output)

# print(x_image.shape)
# print(numerical_inputs.shape)

# Combine the latent space representation with the numerical input
# combined = concatenate([x_image, numerical_inputs])
# print(combined.shape)
# x_combined = Dense(300, kernel_regularizer=l2(0.01), activation='relu')(combined)
# x_combined = Dense(60, kernel_regularizer=l2(0.01), activation='relu')(x_combined)
# outputs = Dense(NUM_OUTPUTS, kernel_regularizer=l2(0.01), activation='linear')(x_combined)


batch is 32
batch is 32
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 480, 640, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 120, 160, 32) 320         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 30, 40, 64)   18496       conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 15, 20, 12

In [6]:
# model = Model([image_inputs, numerical_inputs], outputs)

# model.compile(loss='mse', optimizer='adam', metrics=['mae'])

log_dir = "../logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# history = model.fit([X, inputNums], y, epochs=100, batch_size=32, validation_split=0.2, callbacks=[tensorboard_callback])
# history = model.fit(xy, epochs=100, callbacks=[ValidationCallback(xy_validation), tensorboard_callback] )

# (x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
# mnist_digits = np.concatenate([x_train, x_test], axis=0)
# mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
# vae.compile(optimizer='adam')
# vae.fit(mnist_digits, epochs=30, batch_size=128)
history = vae.fit(xy, epochs=100 )

Epoch 1/100


ValueError: The model cannot be compiled because it has no loss to optimize.

In [ ]:
# Save the model
# model.save('models/conv_model')
model_dir = '../models/conv_model_output_culled'
tf.saved_model.save(model, model_dir)

# Open Tensorboard (actually, run this on the command line on your Terminal)
# %tensorboard --logdir ../logs/fit

In [ ]:
# Load the model
loaded_model = model_load('../models')
loaded_model.fit(x=[X, inputNums], y=y, epochs=100, batch_size=32, validation_split=0.2)